In [4]:
import pandas as pd
from datetime import datetime, timedelta
import uuid

class MassBooking:
    def __init__(self, name=None, mass_intention=None, type_of_mass=None, start_date=None, end_date=None):
        self.name = name.title() if name else None
        self.mass_intention = mass_intention
        self.type_of_mass = type_of_mass.title() if type_of_mass else None
        self.start_date = start_date
        self.end_date = end_date
        self.date_booked = datetime.now()
        self.file_path = 'mass_booked.csv'
        self.columns = ['Booking_ID', 'Name', 'Mass Intention', 'Mass_Type', 'Start Date', 'End Date', 'Date_Booked']
        self.df = pd.DataFrame(columns=self.columns)
        self.load_csv()

    def load_csv(self):
        try:
            self.df = pd.read_csv(
                self.file_path,
                parse_dates=['Start Date', 'End Date', 'Date_Booked']
            )
        except FileNotFoundError:
            self.df = pd.DataFrame(columns=self.columns)

    def save_csv(self):
        self.df.to_csv(self.file_path, index=False)

    def introductory_page(self):
        print('Welcome to St. Cyprian Mass Booking Management System')
        print('Please note each entry is peculiar to a particular mass')
        self.user_introduction = input(
            'What would you like to do?\n'
            'Add a new mass intention (keyword: add)\n'
            'Update a mass intention (keyword: upd)\n'
            'Delete a mass intention (keyword: del)\n'
            'Check booked intentions (keyword: check)\n'
            'Exit the program (keyword: exit)\n> '
        ).lower()

    def validate_dates(self):
        try:
            # parse if string, otherwise assume datetime
            if isinstance(self.start_date, str):
                self.start_date = datetime.strptime(self.start_date, "%Y/%m/%d")
            if isinstance(self.end_date, str):
                self.end_date = datetime.strptime(self.end_date, "%Y/%m/%d")
            if self.start_date > self.end_date:
                raise ValueError("Start date must be before or equal to end date.")
        except ValueError as e:
            print(f'Invalid date: {e}. Please use YYYY/MM/DD and ensure start date <= end date.')
            self.start_date = None
            self.end_date = None
            return False
        return True

    def validate_type_mass(self):
        valid_masses = ['Morning Mass', 'Afternoon Mass', 'Evening Mass', 'First Mass', 'Second Mass', 'Special Mass']
        if isinstance(self.type_of_mass, str):
            self.type_of_mass = self.type_of_mass.title()
        if self.type_of_mass not in valid_masses:
            print(f'Invalid type of mass: {self.type_of_mass}. Valid options are: {valid_masses}')
            return False
        return True

    def generate_booking_id(self):
        return uuid.uuid4().hex[:8]

    def adding_intention(self):
        if not all([self.name, self.mass_intention, self.type_of_mass, self.start_date, self.end_date]):
            print("Cannot add booking: Missing or invalid data.")
            return
        booking_id = self.generate_booking_id()
        new_entry = pd.DataFrame([[
            booking_id,
            self.name,
            self.mass_intention,
            self.type_of_mass,
            self.start_date,
            self.end_date,
            self.date_booked
        ]], columns=self.columns)
        self.df = pd.concat([self.df, new_entry], ignore_index=True)
        self.save_csv()
        print(f"Booking added successfully. Booking ID: {booking_id}")

    def update_intention(self):
        date_filter_str = input("Enter the date the mass was booked (YYYY/MM/DD): ")
        try:
            date_filter = datetime.strptime(date_filter_str, "%Y/%m/%d")
        except ValueError:
            print("Invalid date format. Please use YYYY/MM/DD.")
            return

        name_filter = input("Enter the name used for the booking (optional, press Enter to skip): ").title()
        filtered = self.df[self.df['Date_Booked'].dt.date == date_filter.date()]
        if name_filter:
            filtered = filtered[filtered['Name'] == name_filter]

        if filtered.empty:
            print("No matching records found.")
            return

        print("\nMatching Records:\n", filtered[['Booking_ID', 'Name', 'Mass Intention', 'Start Date', 'End Date']])
        booking_id = input("Enter the Booking ID of the intention to update: ")
        if booking_id not in self.df['Booking_ID'].values:
            print("Invalid Booking ID.")
            return

        idx = self.df.index[self.df['Booking_ID'] == booking_id][0]
        print("Current Record:\n", self.df.loc[idx])

        for col in ['Name', 'Mass Intention', 'Mass_Type', 'Start Date', 'End Date']:
            new_value = input(f"Update {col}? (press Enter to skip): ")
            if new_value:
                if col in ['Start Date', 'End Date']:
                    try:
                        self.df.at[idx, col] = datetime.strptime(new_value, "%Y/%m/%d")
                    except ValueError:
                        print(f"Invalid date for {col}. Skipping update.")
                else:
                    self.df.at[idx, col] = new_value.title() if col in ['Name', 'Mass_Type'] else new_value

        self.save_csv()
        print("Booking updated successfully.")

    def delete_intention(self):
        date_filter_str = input("Enter the date the mass was booked (YYYY/MM/DD): ")
        try:
            date_filter = datetime.strptime(date_filter_str, "%Y/%m/%d")
        except ValueError:
            print("Invalid date format. Please use YYYY/MM/DD.")
            return

        name_filter = input("Enter the name used for the booking (optional, press Enter to skip): ").title()
        filtered = self.df[self.df['Date_Booked'].dt.date == date_filter.date()]
        if name_filter:
            filtered = filtered[filtered['Name'] == name_filter]

        if filtered.empty:
            print("No matching records found.")
            return

        print("\nMatching Records:\n", filtered[['Booking_ID', 'Name', 'Mass Intention', 'Start Date', 'End Date']])
        booking_id = input("Enter the Booking ID of the intention to delete: ")
        if booking_id not in self.df['Booking_ID'].values:
            print("Invalid Booking ID.")
            return

        self.df = self.df[self.df['Booking_ID'] != booking_id]
        self.save_csv()
        print("Booking deleted successfully.")

    def check_bookings(self):
        check_date_str = input("Enter the date to check for mass intentions (YYYY/MM/DD): ")
        try:
            check_date = datetime.strptime(check_date_str, "%Y/%m/%d")
        except ValueError:
            print("Invalid date format. Please use YYYY/MM/DD.")
            return

        self.df['Start Date'] = pd.to_datetime(self.df['Start Date'], errors='coerce')
        self.df['End Date'] = pd.to_datetime(self.df['End Date'], errors='coerce')
        matching = self.df[
            (self.df['Start Date'] <= check_date) &
            (self.df['End Date'] >= check_date)
        ]
        if matching.empty:
            print("No mass intentions booked for that date.")
        else:
            print(f"\nMass intentions for {check_date_str}:")
            print(matching[['Booking_ID', 'Name', 'Mass Intention', 'Mass_Type', 'Start Date', 'End Date']])

    def generate_recurring_dates(self, day_name):
        # start_date and end_date are datetime objects after validate_dates()
        start = self.start_date if isinstance(self.start_date, datetime) else datetime.strptime(self.start_date, "%Y/%m/%d")
        end = self.end_date if isinstance(self.end_date, datetime) else datetime.strptime(self.end_date, "%Y/%m/%d")
        if start > end:
            raise ValueError("Start date must be before or equal to end date.")

        day_lookup = {
            'monday': 0, 'tuesday': 1, 'wednesday': 2, 'thursday': 3,
            'friday': 4, 'saturday': 5, 'sunday': 6
        }
        target = day_lookup.get(day_name.lower())
        if target is None:
            raise ValueError(f"Invalid day: {day_name}")

        dates = []
        current = start
        while current <= end:
            if current.weekday() == target:
                dates.append(current)
            current += timedelta(days=1)
        return dates

    def auto_book_recurring_mass(self):
        self.name = input("Enter your name: ").title()
        self.mass_intention = input("Enter the mass intention: ")
        self.type_of_mass = input("Enter the type of mass (e.g., First Mass): ")
        self.start_date = input("Enter the start date (YYYY/MM/DD): ")
        self.end_date = input("Enter the end date (YYYY/MM/DD): ")
        if not self.validate_dates():
            return
        if not self.validate_type_mass():
            return

        day_name = input("Enter the day of the week for recurring mass (e.g., Sunday): ")
        valid_days = ['monday', 'tuesday', 'wednesday', 'thursday', 'friday', 'saturday', 'sunday']
        if day_name.lower() not in valid_days:
            print("Invalid day. Please enter a valid day of the week.")
            return

        recurring_dates = self.generate_recurring_dates(day_name)
        self.date_booked = datetime.now()
        for date in recurring_dates:
            booking_id = self.generate_booking_id()
            entry = pd.DataFrame([[
                booking_id,
                self.name,
                self.mass_intention,
                self.type_of_mass,
                date,
                date,
                self.date_booked
            ]], columns=self.columns)
            self.df = pd.concat([self.df, entry], ignore_index=True)
            print(f"✔ Booking added: {date.strftime('%Y/%m/%d')} for {self.name} - {self.mass_intention} ({self.type_of_mass}) | Booking ID: {booking_id}")
        self.save_csv()
        print("✅ All recurring masses booked and saved successfully.")


In [6]:
if __name__ == "__main__":
    while True:
        booking = MassBooking()
        booking.introductory_page()

        action = booking.user_introduction.lower()

        if action == 'add':
            recurring = input("Is this a recurring mass? (yes/no): ").strip().lower()
            if recurring == 'yes':
                booking.auto_book_recurring_mass()
            else:
                booking.name = input("Enter your name: ").strip()
                booking.mass_intention = input("Enter the mass intention: ").strip()
                booking.type_of_mass = input("Enter the type of mass (e.g., Morning Mass): ").strip()
                booking.start_date = input("Enter the start date (YYYY/MM/DD): ").strip()
                booking.end_date = input("Enter the end date (YYYY/MM/DD): ").strip()

                booking.validate_dates()
                booking.validate_type_mass()
                booking.adding_intention()

        elif action == 'upd':
            booking.update_intention()

        elif action == 'del':
            booking.delete_intention()

        elif action == 'check':
            booking.check_bookings()

        elif action == 'exit':
            print("Thank you for using the Mass Booking Management System. Goodbye!")
            break

        else:
            print("Invalid option. Please try again.")


Welcome to St. Cyprian Mass Booking Management System
Please note each entry is peculiar to a particular mass


What would you like to do?
Add a new mass intention (keyword: add)
Update a mass intention (keyword: upd)
Delete a mass intention (keyword: del)
Check booked intentions (keyword: check)
Exit the program (keyword: exit)
>  add
Is this a recurring mass? (yes/no):  no
Enter your name:  gdgdg
Enter the mass intention:  fgdfhsfs
Enter the type of mass (e.g., Morning Mass):  Morning Mass
Enter the start date (YYYY/MM/DD):  2025/06/25
Enter the end date (YYYY/MM/DD):  2025/07/22


Booking added successfully. Booking ID: 635f8afd
Welcome to St. Cyprian Mass Booking Management System
Please note each entry is peculiar to a particular mass


What would you like to do?
Add a new mass intention (keyword: add)
Update a mass intention (keyword: upd)
Delete a mass intention (keyword: del)
Check booked intentions (keyword: check)
Exit the program (keyword: exit)
>  check
Enter the date to check for mass intentions (YYYY/MM/DD):  2025/06/29



Mass intentions for 2025/06/29:
   Booking_ID     Name Mass Intention     Mass_Type Start Date   End Date
1    3a63db97  Zimuzor     Gods Mercy       Morning 2025-06-27 2026-09-27
29   635f8afd    gdgdg       fgdfhsfs  Morning Mass 2025-06-25 2025-07-22
Welcome to St. Cyprian Mass Booking Management System
Please note each entry is peculiar to a particular mass


What would you like to do?
Add a new mass intention (keyword: add)
Update a mass intention (keyword: upd)
Delete a mass intention (keyword: del)
Check booked intentions (keyword: check)
Exit the program (keyword: exit)
>  Exit


Thank you for using the Mass Booking Management System. Goodbye!
